In [1]:
#ssh sydney
#setPython3 

import numpy 
import numpy as np
import math
from numpy import loadtxt
import pandas as pd

from scipy.io import arff
data = arff.loadarff('enb.arff')
df = pd.DataFrame(data[0])

covariables = df.iloc[:,0:8].as_matrix()
response = df.iloc[:,8:10].as_matrix()
positions = np.arange(768)

from sklearn.model_selection import train_test_split

covariables_train, covariables_test, response_train, response_test,positions_train,positions_test = train_test_split(covariables, response,positions, test_size=0.20, random_state=42)


##APPRENTISSAGE

def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = numpy.empty(a.shape, dtype=a.dtype)
    shuffled_b = numpy.empty(b.shape, dtype=b.dtype)
    permutation = numpy.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

import tensorflow as tf 




# Import data
# Tensorflow is finicky about shapes, so resize


X_datatrain, X_datatest, targets_train, targets_test = covariables_train, covariables_test, response_train, response_test

#les dimensions de X for training
NbrLignes_train = X_datatrain.shape[0]  ###
NbrColonnes_train= X_datatrain.shape[1] ###

#les dimensions de X for testing
NbrLignes_test = X_datatest.shape[0]  ###
NbrColonnes_test = X_datatest.shape[1] ###
#Y_data = targets 

Y_datatrain = tf.reshape(targets_train, [NbrLignes_train,targets_train.shape[1]])
Y_datatest = tf.reshape(targets_test, [NbrLignes_test,targets_train.shape[1]])

#build the model

# input X: batch_size x NbrColonnes, the first dimension (None) will index the data in the mini-batch
Xfill = tf.placeholder(tf.float32, shape= [None, NbrColonnes_train])
# correct answers will go here
Yfill = tf.placeholder(tf.float32, shape=[None, targets_train.shape[1]]) 
#Let's add some layers

# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

# five layers and their number of neurons (tha last layer has 10 softmax neurons)
L = 200
M = 160
N = 100
O = 30
# Weights initialised with small random values between -0.2 and +0.2
# When using RELUs, make sure biases are initialised with small *positive* values for example 0.1 = tf.ones([K])/10
W1 = tf.Variable(tf.truncated_normal([NbrColonnes_train, L], stddev=0.1))  # 
B1 = tf.Variable(tf.ones([L])/10)
W2 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
B2 = tf.Variable(tf.ones([M])/10)
W3 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
B3 = tf.Variable(tf.ones([N])/10)
W4 = tf.Variable(tf.truncated_normal([N, O], stddev=0.1))
B4 = tf.Variable(tf.ones([O])/10)
W5 = tf.Variable(tf.truncated_normal([O,targets_train.shape[1]], stddev=0.1))
B5 = tf.Variable(tf.zeros([targets_train.shape[1]]))

# The model, with dropout at each layer


Y1 = tf.nn.relu(tf.matmul(Xfill, W1) + B1)
Y1d = tf.nn.dropout(Y1, pkeep)

Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + B2)
Y2d = tf.nn.dropout(Y2, pkeep)

Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + B3)
Y3d = tf.nn.dropout(Y3, pkeep)

Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + B4)
Y4d = tf.nn.dropout(Y4, pkeep)

Y_ = tf.matmul(Y4d, W5) + B5

cross_entropy = tf.losses.mean_squared_error(Yfill, Y_)

# accuracy of the trained model, between 0 (worst) and 1 (best)
#Modification de l'accuracy 

#calcul de l'accuracy, une nouvelle manière: :  

# variable learning rate
lr = tf.placeholder(tf.float32)
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy) 

# init
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init)


max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 100.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

train_loss = []
test_loss = []

# Gradient descent loop for 500 steps
for i in range(3500):
 # Select random minibatch
    j = 1
    ##### shuffling our data::: really important
    X_datasf, Y_datasf = shuffle_in_unison(X_datatrain, Y_datatrain.eval())
    
    ##### defining a decreasing learning rate: 
    # learning rate decay
    learning_rate = 0.0001001329829992624
    #learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
    
    ##### 
    for start, end in zip(range(0, NbrLignes_train, 52), range(20, NbrLignes_train, 52)):

             X_batch,Y_batch = X_datasf[start:end,],Y_datasf[start:end,]
             j +=1
             # Do gradient descent step
             _, loss_val_train = sess.run([train_step, cross_entropy], feed_dict={Xfill: X_batch, Yfill: Y_batch, pkeep: 0.75, lr: learning_rate})
             if i %20==0:    
                 
             
                #if j%20==0:
                 loss_val_test = sess.run(cross_entropy, feed_dict={Xfill: X_datatest, Yfill: Y_datatest.eval(), pkeep: 1.0, lr:  learning_rate})
                 train_loss.append(loss_val_train)
                 
                 test_loss.append(loss_val_test)
                 print('LOSS at step %s: - Train loss value : %f -- Test loss value : %f' % (i, loss_val_train,loss_val_test))
#print(test_loss)
#print(test_loss)
#LOSS at step 420: - Train loss value : 22.385496 -- Test loss value : 9.721767

C:\Users\Trader\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Trader\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
LOSS at step 0: - Train loss value : 1335.923096 -- Test loss value : 1012.382996
LOSS at step 0: - Train loss value : 1247.398438 -- Test loss value : 984.366699
LOSS at step 0: - Train loss value : 1404.847656 -- Test loss value : 958.297913
LOSS at step 0: - Train loss value : 1346.170166 -- Test loss value : 932.659119
LOSS at step 0: - Train loss value : 1096.499512 -- Test loss value : 908.055176
LOSS at step 0: - Train loss value : 914.523132 -- Test loss value : 884.764221
LOSS at step 0: - Train loss value : 792.387024 -- Test loss value : 862.156677
LOSS at step 0: - Train loss value : 1137.547607 -- Test loss value : 839.504761
LOSS at step 0: - Train loss value : 1027.184448 -- Test loss value : 816.919189
LOSS at step 0: - Train loss value : 720.655884 -- Test loss va

LOSS at step 160: - Train loss value : 71.301132 -- Test loss value : 84.918190
LOSS at step 160: - Train loss value : 67.359093 -- Test loss value : 85.012001
LOSS at step 160: - Train loss value : 149.564789 -- Test loss value : 86.387550
LOSS at step 160: - Train loss value : 78.861778 -- Test loss value : 88.274696
LOSS at step 160: - Train loss value : 63.538990 -- Test loss value : 89.681580
LOSS at step 160: - Train loss value : 104.796471 -- Test loss value : 90.372498
LOSS at step 160: - Train loss value : 115.616867 -- Test loss value : 91.376701
LOSS at step 160: - Train loss value : 41.231842 -- Test loss value : 92.567955
LOSS at step 160: - Train loss value : 78.786751 -- Test loss value : 92.975510
LOSS at step 160: - Train loss value : 51.225597 -- Test loss value : 93.390244
LOSS at step 160: - Train loss value : 94.854599 -- Test loss value : 93.498795
LOSS at step 160: - Train loss value : 121.433128 -- Test loss value : 93.714462
LOSS at step 180: - Train loss value

LOSS at step 340: - Train loss value : 82.653976 -- Test loss value : 75.948318
LOSS at step 340: - Train loss value : 45.174496 -- Test loss value : 77.141464
LOSS at step 340: - Train loss value : 73.238373 -- Test loss value : 78.461472
LOSS at step 340: - Train loss value : 48.957169 -- Test loss value : 79.645485
LOSS at step 340: - Train loss value : 23.742502 -- Test loss value : 80.935753
LOSS at step 340: - Train loss value : 50.431263 -- Test loss value : 82.004517
LOSS at step 340: - Train loss value : 41.260582 -- Test loss value : 82.582642
LOSS at step 340: - Train loss value : 31.527851 -- Test loss value : 83.264984
LOSS at step 340: - Train loss value : 34.592903 -- Test loss value : 83.607330
LOSS at step 340: - Train loss value : 96.140793 -- Test loss value : 83.868332
LOSS at step 340: - Train loss value : 31.340902 -- Test loss value : 83.923393
LOSS at step 340: - Train loss value : 37.668221 -- Test loss value : 83.843658
LOSS at step 360: - Train loss value : 5

LOSS at step 520: - Train loss value : 21.739727 -- Test loss value : 65.897743
LOSS at step 520: - Train loss value : 28.280460 -- Test loss value : 65.747513
LOSS at step 520: - Train loss value : 34.139809 -- Test loss value : 65.223671
LOSS at step 520: - Train loss value : 30.864765 -- Test loss value : 65.188553
LOSS at step 520: - Train loss value : 28.713612 -- Test loss value : 65.200073
LOSS at step 520: - Train loss value : 22.955986 -- Test loss value : 65.141281
LOSS at step 520: - Train loss value : 34.075523 -- Test loss value : 64.453529
LOSS at step 520: - Train loss value : 24.154348 -- Test loss value : 63.740055
LOSS at step 520: - Train loss value : 37.200447 -- Test loss value : 62.595131
LOSS at step 520: - Train loss value : 27.083511 -- Test loss value : 61.184925
LOSS at step 520: - Train loss value : 57.856243 -- Test loss value : 60.469322
LOSS at step 520: - Train loss value : 23.925669 -- Test loss value : 59.352577
LOSS at step 540: - Train loss value : 4

...

LOSS at step 2680: - Train loss value : 19.740559 -- Test loss value : 61.829906
LOSS at step 2680: - Train loss value : 30.520166 -- Test loss value : 61.509411
LOSS at step 2680: - Train loss value : 19.435499 -- Test loss value : 60.701057
LOSS at step 2680: - Train loss value : 60.012878 -- Test loss value : 58.949928
LOSS at step 2680: - Train loss value : 13.321558 -- Test loss value : 58.264267
LOSS at step 2680: - Train loss value : 17.682240 -- Test loss value : 56.772087
LOSS at step 2680: - Train loss value : 36.257366 -- Test loss value : 56.986904
LOSS at step 2680: - Train loss value : 27.901245 -- Test loss value : 56.892349
LOSS at step 2680: - Train loss value : 32.651737 -- Test loss value : 58.047600
LOSS at step 2680: - Train loss value : 15.049585 -- Test loss value : 58.733070
LOSS at step 2680: - Train loss value : 16.367504 -- Test loss value : 58.912231
LOSS at step 2680: - Train loss value : 11.539934 -- Test loss value : 59.382294
LOSS at step 2700: - Train l

LOSS at step 2860: - Train loss value : 21.004597 -- Test loss value : 65.910065
LOSS at step 2860: - Train loss value : 23.919456 -- Test loss value : 63.745300
LOSS at step 2860: - Train loss value : 13.518390 -- Test loss value : 60.515770
LOSS at step 2860: - Train loss value : 19.009079 -- Test loss value : 57.134232
LOSS at step 2860: - Train loss value : 33.342163 -- Test loss value : 52.712334
LOSS at step 2860: - Train loss value : 21.469500 -- Test loss value : 48.582642
LOSS at step 2860: - Train loss value : 25.880548 -- Test loss value : 46.282005
LOSS at step 2860: - Train loss value : 32.052624 -- Test loss value : 44.651794
LOSS at step 2860: - Train loss value : 11.624392 -- Test loss value : 43.566807
LOSS at step 2860: - Train loss value : 35.081425 -- Test loss value : 45.132980
LOSS at step 2860: - Train loss value : 15.668263 -- Test loss value : 47.189869
LOSS at step 2860: - Train loss value : 17.448673 -- Test loss value : 49.821011
LOSS at step 2880: - Train l

LOSS at step 3040: - Train loss value : 12.387632 -- Test loss value : 47.653683
LOSS at step 3040: - Train loss value : 13.603763 -- Test loss value : 46.749836
LOSS at step 3040: - Train loss value : 19.145367 -- Test loss value : 45.913643
LOSS at step 3040: - Train loss value : 40.107140 -- Test loss value : 47.054096
LOSS at step 3040: - Train loss value : 29.485569 -- Test loss value : 46.701515
LOSS at step 3040: - Train loss value : 21.007317 -- Test loss value : 47.711040
LOSS at step 3040: - Train loss value : 13.453857 -- Test loss value : 48.225307
LOSS at step 3040: - Train loss value : 25.653784 -- Test loss value : 48.803608
LOSS at step 3040: - Train loss value : 33.159233 -- Test loss value : 47.538315
LOSS at step 3040: - Train loss value : 16.552532 -- Test loss value : 46.039196
LOSS at step 3040: - Train loss value : 20.756109 -- Test loss value : 43.661602
LOSS at step 3040: - Train loss value : 25.087658 -- Test loss value : 41.856445
LOSS at step 3060: - Train l

LOSS at step 3220: - Train loss value : 19.299736 -- Test loss value : 34.540413
LOSS at step 3220: - Train loss value : 12.446665 -- Test loss value : 33.363361
LOSS at step 3220: - Train loss value : 24.078686 -- Test loss value : 33.169090
LOSS at step 3220: - Train loss value : 14.394760 -- Test loss value : 34.176079
LOSS at step 3220: - Train loss value : 23.101004 -- Test loss value : 36.201912
LOSS at step 3220: - Train loss value : 35.228821 -- Test loss value : 39.601658
LOSS at step 3220: - Train loss value : 19.625729 -- Test loss value : 41.357048
LOSS at step 3220: - Train loss value : 18.029886 -- Test loss value : 42.329922
LOSS at step 3220: - Train loss value : 31.637897 -- Test loss value : 45.313972
LOSS at step 3220: - Train loss value : 38.960266 -- Test loss value : 49.818611
LOSS at step 3220: - Train loss value : 15.688542 -- Test loss value : 53.951668
LOSS at step 3220: - Train loss value : 38.036282 -- Test loss value : 55.410183
LOSS at step 3240: - Train l

LOSS at step 3400: - Train loss value : 13.251343 -- Test loss value : 32.817623
LOSS at step 3400: - Train loss value : 10.271906 -- Test loss value : 32.518284
LOSS at step 3400: - Train loss value : 13.835001 -- Test loss value : 32.858505
LOSS at step 3400: - Train loss value : 33.008125 -- Test loss value : 32.773952
LOSS at step 3400: - Train loss value : 13.732285 -- Test loss value : 32.961529
LOSS at step 3400: - Train loss value : 18.896498 -- Test loss value : 33.330593
LOSS at step 3400: - Train loss value : 47.369511 -- Test loss value : 35.206963
LOSS at step 3400: - Train loss value : 11.087680 -- Test loss value : 36.934151
LOSS at step 3400: - Train loss value : 27.369324 -- Test loss value : 38.100719
LOSS at step 3400: - Train loss value : 17.331944 -- Test loss value : 38.944118
LOSS at step 3400: - Train loss value : 22.542967 -- Test loss value : 39.373207
LOSS at step 3400: - Train loss value : 27.420185 -- Test loss value : 40.852623
LOSS at step 3420: - Train l

# Let us compute the aRRMSE

In [2]:
learning_rate = 0.0001001329829992624

Y_testpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatest, tf.float32).eval(), Yfill: tf.cast(Y_datatest, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})
#numpy.savetxt('Y_testpred_with_stock_%s_and_ai_%s_modelcut_%s_vartimeframe_%s.txt'%(stock, ai, cut, vartimeframe), Y_testpred, fmt='%f')
Y_trainpred = Y_.eval(feed_dict={Xfill: tf.cast(X_datatrain, tf.float32).eval(), Yfill: tf.cast(Y_datatrain, tf.float32).eval(), pkeep: 1.0, lr: learning_rate})


In [3]:
Y_testpred

array([[11.587264 , 13.373474 ],
       [ 9.790453 , 11.436064 ],
       [25.63427  , 27.450064 ],
       [34.71381  , 36.875187 ],
       [11.583683 , 13.372759 ],
       [21.55212  , 23.114904 ],
       [21.085426 , 22.734278 ],
       [23.207905 , 24.937716 ],
       [11.52074  , 13.299318 ],
       [23.718466 , 25.483442 ],
       [11.864896 , 13.687683 ],
       [24.03741  , 25.750177 ],
       [23.777927 , 25.546131 ],
       [ 7.715579 ,  9.323542 ],
       [12.071358 , 13.914564 ],
       [24.115993 , 25.848465 ],
       [25.146633 , 26.911602 ],
       [ 7.896756 ,  9.519569 ],
       [ 8.363048 , 10.005705 ],
       [24.016376 , 25.728445 ],
       [24.400135 , 26.037891 ],
       [33.776604 , 35.91008  ],
       [ 8.689945 , 10.238527 ],
       [29.890512 , 31.870201 ],
       [11.283282 , 13.040041 ],
       [29.94162  , 31.925423 ],
       [27.903063 , 29.826508 ],
       [24.2457   , 25.878471 ],
       [11.048286 , 12.776697 ],
       [11.086009 , 12.820864 ],
       [ 7

In [4]:
from numpy import linalg as LA

In [5]:

aRRMSE_list = []

for i in range(Y_testpred.shape[1]) :
    
    Errors_test =  Y_testpred[:,i] - response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    ytrain_mean_i = np.tile(Y_trainpred[:,i].mean(axis = 0), (covariables_test.shape[0], 1))

    Errors_relative = ytrain_mean_i - response_test[:,i]

    Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%Final_SPNNR_aRRMSE)

    aRRMSE_list.append(Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 4.166283
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 4.553224


In [6]:
np.mean(aRRMSE_list)

4.359753384953411

In [9]:
#Taking aRRMSE on PCA or inverted PCA. ?

# Check if old way calculation is wrong

In [10]:
Errors_test =  Y_testpred - response_test


print('<*********************************Let us check the aRRMSE*********************************>')

ytrain_mean = np.tile(Y_trainpred.mean(axis = 0), (response_test.shape[0], 1))

Errors_relative = ytrain_mean - response_test

Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(Errors_test)**2)/(LA.norm(Errors_relative)**2))  

print("Final_Partition aRRMSE is : %f"%Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final_Partition aRRMSE is : 82.647642


*** The answer is yes ***

# Checking True aRRMSE of S-SPNNR on Slump

In [11]:
spnnr_covariables_train, spnnr_covariables_test, spnnr_response_train, spnnr_response_test,spnnr_positions_train,spnnr_positions_test = train_test_split(covariables, response,positions, test_size=0.33, random_state=42)

In [12]:
spnnr_Y_testpred =np.loadtxt('All_Y_test_predictions.out', delimiter=',' )
spnnr_Y_trainpred =np.loadtxt('All_Y_train_predictions.out', delimiter=',' )

In [10]:
spnnr_aRRMSE_list = []

for i in range(spnnr_Y_testpred.shape[1]) :
    
    spnnr_Errors_test =  spnnr_Y_testpred[:,i] - spnnr_response_test[:,i]
    print('<*********************************Let us check the aRRMSE*********************************>')

    spnnr_ytrain_mean_i = np.tile(spnnr_Y_trainpred[:,i].mean(axis = 0), (spnnr_covariables_test.shape[0], 1))

    spnnr_Errors_relative = spnnr_ytrain_mean_i - spnnr_response_test[:,i]

    spnnr_Final_SPNNR_aRRMSE = 100*np.sqrt((LA.norm(spnnr_Errors_test)**2)/(LA.norm(spnnr_Errors_relative)**2))  

    print("Final i-th aRRMSE is : %f"%spnnr_Final_SPNNR_aRRMSE)

    spnnr_aRRMSE_list.append(spnnr_Final_SPNNR_aRRMSE)

<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 9.212108
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 17.784301
<*********************************Let us check the aRRMSE*********************************>
Final i-th aRRMSE is : 18.862991


In [11]:
np.mean(spnnr_aRRMSE_list)

15.286466834678842